In [8]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-crmmn')

In [9]:
m = geemap.Map(center=[11.8, 77.8], zoom=7)
m

Map(center=[11.8, 77.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [10]:
mettur_geom = ee.Geometry.Point([77.8, 11.8]).buffer(25000)
cauvery_bbox = ee.Geometry.Rectangle([74.5, 10.5, 79.5, 13.5])
m.add_layer(mettur_geom, {'color': 'red'}, 'Mettur Buffer')
m

Map(bottom=15602.0, center=[11.8, 77.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

In [11]:
water = (
    ee.ImageCollection('JRC/GSW1_3/YearlyHistory')
    .filterBounds(cauvery_bbox)
    .select('waterClass')
    .filterDate('2000-01-01', '2020-12-31')
)

In [12]:
water_list = water.toList(water.size())
sampled_water = ee.ImageCollection([
    ee.Image(water_list.get(i)) for i in range(0, water.size().getInfo(), 2)
])
sampled_times = sampled_water.aggregate_array('system:index').getInfo()

In [13]:
vis_params = {
    'min': 0,
    'max': 2,
    'palette': ['000000', 'ffffff', '0000ff']  # Black, white, blue
}

In [14]:
m.ts_inspector(
    left_ts=sampled_water,
    right_ts=sampled_water,
    left_names=sampled_times,
    right_names=sampled_times,
    left_vis=vis_params,
    right_vis=vis_params,
    width="80px"
)
m

Map(bottom=15602.0, center=[11.8, 77.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…

In [15]:
states = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq("ADM0_NAME", "India"))
states_styled = states.style(**{"color": "black", "fillColor": "00000000",'width':1})
m.add_layer(states_styled, {}, "State Boundaries")
m

Map(bottom=244141.0, center=[12.468089693756074, 76.54219914262033], controls=(WidgetControl(options=['positio…

In [16]:
basins = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_5")
cauvery_basins = basins.filterBounds(cauvery_bbox)

In [17]:
m.add_layer(cauvery_basins.style(**{"color": "red", "width": 1,'fillColor':'#00000000'}), {}, "Cauvery Basin")
m

Map(bottom=244141.0, center=[12.468089693756074, 76.54219914262033], controls=(WidgetControl(options=['positio…

In [42]:
cauvery_label_point = ee.Geometry.Point([78.0, 12.2])
cauvery_label = ee.Feature(cauvery_label_point, {'label': 'Cauvery Basin'})
m.add_labels(data=ee.FeatureCollection([cauvery_label]), column='label', font_size='12pt', font_color='blue')

In [43]:
gif_vis_params = {
    'bands': ['waterClass'],
    'min': 0,
    'max': 2,
    'palette': ['000000', 'ffffff', '0000ff']
}

In [44]:
 video_args={
        'dimensions': 768,
        'region': cauvery_bbox,
        'framesPerSecond': 2,
        'crs': 'EPSG:4326',
        'bands': ['waterClass'],
        'min': 0,
        'max': 2,
        'palette': ['000000', 'ffffff', '0000ff']
    }

In [45]:
geemap.download_ee_video(
    sampled_water,
    video_args,
    out_gif='cauvery_timelapse.gif',
)

Generating URL...
Please wait ...
The GIF image has been saved to: /content/cauvery_timelapse.gif


In [46]:
geemap.show_image('/content/cauvery_timelapse.gif')

Output()